In [1]:
import os

from dotenv import load_dotenv
from hubspot import HubSpot
import openai

import json
import logging
import copy


In [2]:
load_dotenv()
hub_api = os.getenv("HUB_API")
hs_client = HubSpot(access_token=hub_api)

In [8]:
hs_client.crm.contacts.get_all()

[{'archived': False,
  'archived_at': None,
  'associations': None,
  'created_at': datetime.datetime(2023, 7, 17, 23, 56, 30, 160000, tzinfo=tzutc()),
  'id': '101',
  'properties': {'createdate': '2023-07-17T23:56:30.160Z',
                 'email': 'luisg@embraer.com.br',
                 'firstname': 'Luís',
                 'hs_object_id': '101',
                 'lastmodifieddate': '2023-07-17T23:58:56.326Z',
                 'lastname': 'Gonçalves'},
  'properties_with_history': None,
  'updated_at': datetime.datetime(2023, 7, 17, 23, 58, 56, 326000, tzinfo=tzutc())},
 {'archived': False,
  'archived_at': None,
  'associations': None,
  'created_at': datetime.datetime(2023, 7, 18, 0, 6, 7, 529000, tzinfo=tzutc()),
  'id': '151',
  'properties': {'createdate': '2023-07-18T00:06:07.529Z',
                 'email': 'tgoyer@apple.com',
                 'firstname': 'Tim',
                 'hs_object_id': '151',
                 'lastmodifieddate': '2023-07-18T00:09:25.543Z',
       

In [9]:
SUPPORTED_ASSOCIATIONS = [
    "deal_to_company",
    "deal_to_contact",
    "deal_to_line_item",
    "company_to_contact",
    "company_to_deal",
    "contact_to_company",
    "contact_to_deal",
    "meeting_event_to_deal",
    "deal_to_task",
    "deal_to_meeting_event",
    "deal_to_note",
    "deal_to_call",
    "call_to_deal",
    "note_to_deal",
    "task_to_deal",
]

load_dotenv()
hub_api = os.getenv("HUB_API")
hs_client = HubSpot(access_token=hub_api)


def extract_associations(raw_associations):
    associations = {}
    for entity_type in raw_associations:
        associations[entity_type] = []
        for association_data in raw_associations[entity_type].results:
            if association_data.type not in SUPPORTED_ASSOCIATIONS:
                continue
            associations[entity_type].append(association_data.id)
    return associations


def flatten_contact(raw_response):
    return {
        "id": raw_response.id,
        "email": raw_response.properties["email"],
        "firstname": raw_response.properties["firstname"],
        "lastname": raw_response.properties["lastname"],
        "associations": extract_associations(raw_response.associations),
    }


def flatten_company(raw_response):
    return {
        "id": raw_response.id,
        "domain": raw_response.properties["domain"],
        "name": raw_response.properties["name"],
        "associations": extract_associations(raw_response.associations),
    }


def flatten_deal(raw_response):
    return {
        "id": raw_response.id,
        "dealname": raw_response.properties["dealname"],
        "dealstage": raw_response.properties["dealstage"],
        "amount": raw_response.properties["amount"],
        "closedate": raw_response.properties["closedate"],
        "createdate": raw_response.properties["createdate"],
        "lastmodifeddate": raw_response.properties["hs_lastmodifieddate"],
        "hubspot_owner_id": raw_response.properties["hubspot_owner_id"],
        "associations": extract_associations(raw_response.associations),
    }


def flatten_owner(raw_response):
    return {
        "id": raw_response.id,
        "first_name": raw_response.first_name,
        "last_name": raw_response.last_name,
        "email": raw_response.email,
    }


def flatten_products(raw_response):
    return {
        "id": raw_response.id,
        "name": raw_response.properties["name"],
        "quantity": raw_response.properties["quantity"],
        "amount": raw_response.properties["amount"],
    }


def flatten_task(raw_task):
    return {
        "id": raw_task.id,
        "owner_id": raw_task.properties["hubspot_owner_id"],
        "subject": raw_task.properties["hs_task_subject"],
        "status": raw_task.properties["hs_task_status"],
        "priority": raw_task.properties["hs_task_priority"],
        "type": raw_task.properties["hs_task_type"],
        "body": raw_task.properties["hs_task_body"]
    }


def flatten_note(raw_note):
    # Modify based on actual structure of a note object
    return {
        "id": raw_note.id,
        "body": raw_note.properties["hs_note_body"],
        "owner_id": raw_note.properties["hubspot_owner_id"]
    }


def flatten_call(raw_call):
    return {
        "id": raw_call.id,
        "body": raw_call.properties["hs_call_body"],
        "direction": raw_call.properties["hs_call_direction"],
        "disposition": raw_call.properties["hs_call_disposition"],
        "duration": raw_call.properties["hs_call_duration"],
        "status": raw_call.properties["hs_call_status"],
        "title": raw_call.properties["hs_call_title"]
    }


def flatten_meeting(raw_meeting):
    return {
        "id": raw_meeting.id,
        "title": raw_meeting.properties["hs_meeting_title"],
        "body": raw_meeting.properties["hs_meeting_body"],
        "location": raw_meeting.properties["hs_meeting_location"]
    }


def get_activities():
    # print("listing activities")

    tasks = [
        flatten_task(task)
        for task in hs_client.crm.objects.get_all(
            object_type="tasks",
            associations=["deals"],
            properties=[
                "hubspot_owner_id",
                "hs_task_subject",
                "hs_task_status",
                "hs_task_priority",
                "hs_task_type",
                "hs_task_body",
            ],
        )
    ]

    notes = [
        flatten_note(note)
        for note in hs_client.crm.objects.get_all(
            object_type="notes",
            associations=["deals"],
            properties=["hs_note_body", "hubspot_owner_id"],
        )
    ]

    calls = [
        flatten_call(call)
        for call in hs_client.crm.objects.get_all(
            object_type="calls",
            associations=["deals"],
            properties=[
                "hs_call_body",
                "hs_call_direction",
                "hs_call_disposition",
                "hs_call_duration",
                "hs_call_status",
                "hs_call_title",
            ],
        )
    ]

    meetings = [
        flatten_meeting(meeting)
        for meeting in hs_client.crm.objects.get_all(
            object_type="meetings",
            associations=["deals"],
            properties=["hs_meeting_title", "hs_meeting_body", "hs_meeting_location"],
        )
    ]

    return {"tasks": tasks, "notes": notes, "calls": calls, "meetings": meetings}


def get_all_contacts():
    # print("listing contacts")
    response = []
    for raw_response in hs_client.crm.contacts.get_all(
            associations=["companies", "deals"]
    ):
        response.append(flatten_contact(raw_response))
    return response


def get_all_companies():
    # print("listing companies")
    response = []
    for raw_response in hs_client.crm.companies.get_all(
            associations=["contacts", "deals"]
    ):
        response.append(flatten_company(raw_response))
    return response


def get_products():
    response = []
    for raw_response in hs_client.crm.line_items.get_all(
            properties=["name", "quantity", "amount"]
    ):
        response.append(flatten_products(raw_response))
    return response


def get_deal_owner():
    # print("listing owners")
    response = []
    for raw_response in hs_client.crm.owners.get_all():
        response.append(flatten_owner(raw_response))
    return response


def get_all_deals():
    # print("listing deals")
    response = []
    for raw_response in hs_client.crm.deals.get_all(
            associations=[
                "companies",
                "contacts",
                "line_items",
                "calls",
                "meetings",
                "tasks",
                "notes",
                "tickets",
            ],
            properties=["hubspot_owner_id", "amount", "dealname", "dealstage", "closedate"],
    ):
        response.append(flatten_deal(raw_response))
    return response


def match_activities_to_deals():
    deals = get_all_deals()
    activities = get_activities()

    activity_map = {}
    for activity_type, activity_list in activities.items():
        for activity in activity_list:
            activity_map[activity['id']] = (activity_type, activity)

    deal_activities = {}

    for deal in deals:
        if not isinstance(deal, dict):
            print(f"Unexpected item in deals: {deal}")
            continue

        deal_id = deal.get('id')
        dealname = deal.get('dealname')
        associations = deal.get('associations', {})

        matched_activities = {
            'tasks': [],
            'notes': [],
            'calls': [],
            'meetings': []
        }

        for associated_type, associated_ids in associations.items():
            for associated_id in associated_ids:
                if associated_id in activity_map:
                    activity_type, activity_data = activity_map[associated_id]
                    matched_activities[activity_type].append(activity_data)

        deal_activities[dealname] = matched_activities

    return deal_activities


In [10]:
print(get_all_deals())

[{'id': '14202296949', 'dealname': 'Embraer SA - New Deal', 'dealstage': 'qualifiedtobuy', 'amount': '100000', 'closedate': '2024-01-01T00:58:37.354Z', 'createdate': '2023-07-17T23:58:55.432Z', 'lastmodifeddate': '2023-10-06T14:00:12.998Z', 'hubspot_owner_id': '461935514', 'associations': {'companies': ['16580143418'], 'calls': ['40877702012'], 'contacts': ['101']}}, {'id': '14202419464', 'dealname': 'Apple Inc. - New Deal', 'dealstage': 'presentationscheduled', 'amount': '100000', 'closedate': '2023-10-17T00:06:35.863Z', 'createdate': '2023-07-18T00:06:46.974Z', 'lastmodifeddate': '2023-10-11T16:19:11.849Z', 'hubspot_owner_id': '482201636', 'associations': {'companies': ['16580143520'], 'notes': ['37468062483'], 'tickets': [], 'calls': ['37468331624'], 'contacts': ['151']}}, {'id': '14202504861', 'dealname': 'Google - New Deal', 'dealstage': 'qualifiedtobuy', 'amount': '300000', 'closedate': '2023-08-18T00:15:41.865Z', 'createdate': '2023-07-18T00:15:51.422Z', 'lastmodifeddate': '2023

In [11]:
logger = logging.getLogger(__name__)
openai.api_key = os.getenv("OPENAI_API_KEY")

def call_function(available_functions, response_message, messages):
    function_name = response_message["function_call"]["name"]
    arguments = response_message["function_call"]["arguments"]
    logger.info(
        f"LLM wants to call {function_name} function with arguments: {arguments}"
    )
    function_to_call = available_functions.get(function_name)
    if function_to_call is None:
        logger.error(
            f"LLM wanted to call function {function_name} that is not available"
        )
        messages.append(
            {
                "role": "user",
                "content": f"There is no function such: {function_name}. Available functions are: {','.join(available_functions.keys())}",
            }
        )
        return
    function_args = json.loads(response_message["function_call"]["arguments"])
    function_response = function_to_call(**function_args)
    function_response = json.dumps(function_response)
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )
    logger.info(
        f"{function_name} for arguments {arguments} returned {function_response}"
    )


def chat(
    prompt, messages, available_functions, functions, max_turns=15, agent_name="agent"
):
    logger.info(f"Entering chat function for {agent_name} and prompt:\n {prompt}")
    messages_new = copy.deepcopy(messages)
    messages_new.append({"role": "user", "content": prompt})
    turns = 0
    while True:
        response = openai.ChatCompletion.create(
            model="gpt-4-1106-preview",
            messages=messages_new,
            functions=functions,
            function_call="auto",
            temperature=0.00000001,
        )
        response_message = response["choices"][0]["message"]
        message_content = response["choices"][0]["message"]["content"]
        if message_content:
            logger.info(f"LLM responded with message: {message_content}\n")
        messages_new.append(response_message)
        if response_message.get("function_call"):
            call_function(available_functions, response_message, messages_new)
        else:
            logger.info(f"chat function for {agent_name} returned:\n {message_content}")
            break
        turns += 1
        if turns > max_turns:
            raise Exception("Reached max number of turns to LLM for single user query")
    messages[:] = messages_new
    return (
        response["choices"][0]["message"]["content"],
        response["usage"]["total_tokens"],
    )


In [12]:
class HubspotAgent:
    SYSTEM_PROMPT = """
    You are a virtual sales team assistant specializing in Hubspot-related queries.
    Your expertise lies in leveraging the Hubspot API to fetch data.
    Hubspot contains various entities, interconnected through associations.
    Your primary role is to exploit these associations for precise data filtration and aggregation.
    Avoid generating new code; instead, utilize pre-existing functions.
    When tasked with filtering data from Hubspot, first retrieve the complete dataset and then extract the specific 
    segments as instructed by the user.
    Please note that, closewon to the deal stage is the only closed and successful deal, the others remain open and 
    should not be taken into final consideration in regards revenue. 
    ALL OPEN DEALS are important to predict forecast revenue!!! 
    When I am asking about the forecast - take under consideration open deals and the price (not closed won!)
    Remember to match the deals with appropriate deal owner and its activities!
    *** If Hubspot function is not available - just list the information. DO NOT INVENT FUNCTIONS ***
    *** DO NOT GENERATE ANY CODE! JUST PROVIDE ANSWER TO THE QUESTION! ***
    *** Answer the question DIRECTLY. DO NOT provide or mention any plan or steps on how to solve the problem. ***
    ACTIVITIES include tasks, meetings, calls, notes and tickets!!
    *** ALWAYS MATCH ACTIVITIES WITH SPECIFIC DEAL!!! NEVER PRINT ALL THE ACTIVITIES FOR ALL THE DEALS!!***
    """

    PROMPT_TEMPLATE = """
        Please follow the approach below during solving the problem:
            1. Let’s first understand the problem and devise a plan to solve the problem.
               Please make the plan the minimum number of steps and the minimum number of functions calls required
               to accurately complete the task.
            2. Output the plan to the user and carry out the plan and solve the problem step by step
            ''' DO NOT PROVIDE THE PLAN - JUST EXECUTE AND GIVE ME FINAL OUTPUT '''
            Problem: {}
    """

    def __init__(self):
        self.available_functions = {
            "get_all_companies": get_all_companies,
            "get_all_deals": get_all_deals,
            "get_all_contacts": get_all_contacts,
            "get_deal_owner": get_deal_owner,
            "get_products": get_products,
            "get_activities": get_activities,
            "match_activities_to_deals": match_activities_to_deals,
        }
        self.functions = [
            {
                "name": "get_all_companies",
                "description": "Returns all companies available in Hubspot.",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": [],
                },
            },
            {
                "name": "get_all_contacts",
                "description": "Returns all contacts available in Hubspot.",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": [],
                },
            },
            {
                "name": "get_all_deals",
                "description": "Returns all deals available in Hubspot.",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": [],
                },
            },
            {
                "name": "get_deal_owner",
                "description": "Returns all owners available in Hubspot.",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": [],
                },
            },
            {
                "name": "get_products",
                "description": "Returns products associated with a deal in Hubspot.",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": [],
                },
            },
            {
                "name": "get_activities",
                "description": "Returns products activities with a deal in Hubspot.",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": [],
                },
            },
            {
                "name": "match_activities_to_deals",
                "description": "Matches activities to specific deals and returns the mapped results.",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": [],
                },
            },
        ]
        self.messages = [{"role": "system", "content": self.SYSTEM_PROMPT}]

    def chat(self, query):
        prompt = self.PROMPT_TEMPLATE.format(query)
        response, _ = chat(
            prompt,
            self.messages,
            self.available_functions,
            self.functions,
            agent_name="hubspot_agent",
        )
        return response


def ask_hubspot_agent(query):
    print("asking Hubspot Agent")
    return HubspotAgent().chat(query)

In [ ]:
if __name__ == "__main__":
    while True:
        query = input("Enter your query (or type 'exit' to end): ")
        if query.lower() == "exit":
            break
        response = ask_hubspot_agent(query)
        print("Agent:", response)